In [1]:
from os.path import basename
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.cli import LightningArgumentParser

from pl_modules import PlLanguageModelForSequenceOrdering

In [2]:
parser = LightningArgumentParser()
group = parser.add_mutually_exclusive_group()
group.add_argument("--run.run_name", type=str, default="test")
group.add_argument("--run.seed", type=int, default=0)
group.add_argument("--run.train_batch_size", type=int, default=8)
group.add_argument("--run.val_batch_size", type=int, default=16)

parser.add_lightning_class_args(ModelCheckpoint, "checkpoint")
parser.add_class_arguments(TensorBoardLogger, nested_key="tensorboard")
parser.add_lightning_class_args(Trainer, "trainer")
parser = PlLanguageModelForSequenceOrdering.add_model_specific_args(parser)

In [8]:
args = parser.parse_args(["--tensorboard.save_dir", "blub"])
args.get("run", {})

{'run_name': 'test', 'seed': 0, 'train_batch_size': 8, 'val_batch_size': 16}

In [5]:
dir(ModelCheckpoint)

['CHECKPOINT_JOIN_CHAR',
 'CHECKPOINT_NAME_LAST',
 'FILE_EXTENSION',
 'STARTING_VERSION',
 '_ModelCheckpoint__init_ckpt_dir',
 '_ModelCheckpoint__init_monitor_mode',
 '_ModelCheckpoint__init_triggers',
 '_ModelCheckpoint__resolve_ckpt_dir',
 '_ModelCheckpoint__validate_init_configuration',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_del_model',
 '_format_checkpoint_name',
 '_get_metric_interpolated_filepath_name',
 '_is_valid_monitor_key',
 '_monitor_candidates',
 '_save_last_checkpoint',
 '_save_model',
 '_save_none_monitor_checkpoint',
 '_save_top_k_checkpoint',
 '_should_skip_saving_checkpoint',
 '_update_bes

In [9]:
class Test:
    def __init__(self, data):
        self.data = data

    def __dict__(self):
        return self.data

In [10]:
t = Test({"hallo": 1})

In [11]:
dict(**t)

TypeError: dict() argument after ** must be a mapping, not Test

In [15]:
d = AttrDict({"hallo": 1})

In [16]:
d.hallo

1